In [ ]:
# Code taken from Week 2 Session Activity. 

In [54]:
# installing relevantlibraries
!pip install pandas
!pip install sklearn


In [55]:
!pip3 install -U scikit-learn scipy matplotlib

In [56]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kzaromytidou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
import pandas as pd 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words as stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
import re
from sklearn.decomposition import TruncatedSVD

In [58]:
# #Reading the CSV file into a pandas dataframe
data_from_csv = pd.read_csv('data/Putin_Tweets.csv', encoding='utf-8') 
data_from_csv.sample(5) #show a  sample  of 5 rows

,UserScreenName,UserName,...,Image link,Tweet URL
3828,susan D,@footballsuz1564,...,[],https://twitter.com/footballsuz1564/status/150...
8959,Jeff the,@DemannFunk,...,[],https://twitter.com/DemannFunk/status/15419221...
5964,Chip Taylor,@ChipTaylor17,...,[],https://twitter.com/ChipTaylor17/status/152055...
4907,Senate Foreign Relations Committee,@SFRCdems,...,['https://pbs.twimg.com/profile_images/1061915...,https://twitter.com/SFRCdems/status/1512935410...
9208,Salvatore #OndaCivica #FBPEGlobal,@salfasanop,...,['https://pbs.twimg.com/profile_images/4654931...,https://twitter.com/salfasanop/status/15437287...


In [59]:
#showing how many rows and columns are included in the data file. 
np.shape(data_from_csv) 

(10737, 11)

In [41]:
#firstly change an existing column with the name 'Text to 'Source' as there is another column that better ideintfies as text. 
df1 = data_from_csv.rename(columns={'Text': 'source'})
df1

,UserScreenName,UserName,...,Image link,Tweet URL
0,Free Syrian Memes,@FreeSyrianMemes,...,['https://pbs.twimg.com/media/FIDMAkeXsAcBOwG?...,https://twitter.com/FreeSyrianMemes/status/147...
1,ciciu,@_ciciulena,...,[],https://twitter.com/_ciciulena/status/14773797...
2,TOM,@lonestarmoocher,...,['https://pbs.twimg.com/media/FICgVy4VgAMHEqH?...,https://twitter.com/lonestarmoocher/status/147...
3,Tom London,@TomLondon6,...,[],https://twitter.com/TomLondon6/status/14774092...
4,Ukemonde,@ukemonde,...,['https://pbs.twimg.com/media/FIDHb0SXMAcqHoh?...,https://twitter.com/ukemonde/status/1477399960...
...,...,...,...,...,...
10732,RoCo STOP PUTIN!,@RoCoGB,...,['https://pbs.twimg.com/profile_images/1042802...,https://twitter.com/RoCoGB/status/154833229582...
10733,Stefan Resists Putin,@StefanK1117,...,['https://pbs.twimg.com/profile_images/1263428...,https://twitter.com/StefanK1117/status/1548190...
10734,Comrade Misty is Putin’s Buddy,@SarcasmStardust,...,['https://pbs.twimg.com/card_img/1548668878741...,https://twitter.com/SarcasmStardust/status/154...
10735,Dr Putin parody russian president,@TheSigmaMaleDoc,...,['https://pbs.twimg.com/profile_images/1173895...,https://twitter.com/TheSigmaMaleDoc/status/154...


In [60]:
#now rename the column 'Embedded_text' to 'text' to better use in the process. 
df = df1.rename(columns={'Embedded_text': 'text'})
df

,UserScreenName,UserName,...,Image link,Tweet URL
0,Free Syrian Memes,@FreeSyrianMemes,...,['https://pbs.twimg.com/media/FIDMAkeXsAcBOwG?...,https://twitter.com/FreeSyrianMemes/status/147...
1,ciciu,@_ciciulena,...,[],https://twitter.com/_ciciulena/status/14773797...
2,TOM,@lonestarmoocher,...,['https://pbs.twimg.com/media/FICgVy4VgAMHEqH?...,https://twitter.com/lonestarmoocher/status/147...
3,Tom London,@TomLondon6,...,[],https://twitter.com/TomLondon6/status/14774092...
4,Ukemonde,@ukemonde,...,['https://pbs.twimg.com/media/FIDHb0SXMAcqHoh?...,https://twitter.com/ukemonde/status/1477399960...
...,...,...,...,...,...
10732,RoCo STOP PUTIN!,@RoCoGB,...,['https://pbs.twimg.com/profile_images/1042802...,https://twitter.com/RoCoGB/status/154833229582...
10733,Stefan Resists Putin,@StefanK1117,...,['https://pbs.twimg.com/profile_images/1263428...,https://twitter.com/StefanK1117/status/1548190...
10734,Comrade Misty is Putin’s Buddy,@SarcasmStardust,...,['https://pbs.twimg.com/card_img/1548668878741...,https://twitter.com/SarcasmStardust/status/154...
10735,Dr Putin parody russian president,@TheSigmaMaleDoc,...,['https://pbs.twimg.com/profile_images/1173895...,https://twitter.com/TheSigmaMaleDoc/status/154...


In [ ]:
# Applying LSA. following the code from "NLP Week 2.2-Lecture.ipynb" notebook 

**Now, let's apply LSA.**

Grab some code from the "NLP Week 2.2-Lecture.ipynb" notebook to do the following:

* Create count vectors for your corpus, using a tokenizer of your choice (e.g., possibly Louis' custom tokenizer, or perhaps another one you think will be good for this task)
* Create tf-idf vectors from the count vectors
* Centre the vectorized documents by subtracting the mean
* Use TruncatedSVD to compute the LSA topic vectors. Use 10 topics and 30 iterations to start with.
* Explore the results.
  * Do the topics seem to correspond to movie genres, to types of reviews, or anything else? 
  * Do the topic weights for individual reviews seem to make sense to you?
* Try changing the number of topics to fewer (e.g., 3?) or more (e.g., 20) and explore how this changes the results.
* Optional bonus: Can you write some code that allows you to construct a query for this corpus? It should use cosine similarity to find the movie review(s) that best match a text string you provide.

In [43]:
#Creating count vectors for the corpus. 
lem = WordNetLemmatizer()
def my_tokeniser(doc):
    #Split on spaces
    tokens = re.split(r'[-\s.,;!?]+', doc)
    return [lem.lemmatize(t.lower()) for t in tokens if not t in stop_words.ENGLISH_STOP_WORDS]

In [44]:
#  Using a tokenizer
tfidf_vectoriser = TfidfVectorizer(tokenizer=my_tokeniser)

In [45]:
# Create tf-idf vectors from the count vectors
tfidf = tfidf_vectoriser.fit_transform(df["text"])
#Save list of unique tokens (vocab) for later
vocab = tfidf_vectoriser.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf.todense(), columns = vocab)
print(tfidf.todense().shape)

/Users/kzaromytidou/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(10737, 40394)


In [46]:
#Centring the vectorized documents by subtracting the mean
tfidf_df = tfidf_df - tfidf_df.mean()

In [47]:
#Selecting the amount of topics
num_topics = 5
pd.options.display.max_columns=num_topics
labels = ['topic{}'.format(i) for i in range(num_topics)]

In [48]:
labels

['topic0', 'topic1', 'topic2', 'topic3', 'topic4']

In [49]:
#Calculating  the topics
svd = TruncatedSVD(n_components = num_topics, n_iter = 100) 
svd_topic_vectors = svd.fit_transform(tfidf_df.values)

In [50]:
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, columns=labels)
svd_topic_vectors_df.sample(10)

,topic0,topic1,topic2,topic3,topic4
8635,-0.016524,0.037609,-0.016443,-0.075015,0.001281
9155,-0.096730,-0.034389,-0.000352,-0.022098,-0.008409
4136,-0.025853,0.011770,-0.035068,-0.035704,0.037031
6854,0.055518,-0.022813,0.115357,0.044891,0.032662
2261,0.082534,-0.006170,0.072877,-0.010984,0.027822
5809,0.045710,-0.049622,-0.015528,0.026464,0.008322
7217,0.011676,-0.045691,-0.027386,-0.015149,-0.017180
6345,0.075423,0.058743,-0.032145,0.111943,-0.026938
5653,-0.004764,-0.075769,-0.026904,0.069425,-0.012882
2007,-0.004880,0.025897,0.008604,-0.048398,-0.004206


In [51]:
#Calculating and printing how  much does each topic apply to each token
topic_weights = pd.DataFrame(svd.components_.T, index=vocab, columns=labels)
topic_weights.sample(20) 

,topic0,topic1,topic2,topic3,topic4
theft,0.001578,0.000934,0.001068,-0.001402,0.000585
@atltuck,0.000102,0.000681,0.000158,-0.000122,0.000152
checker,-0.000712,0.000125,-0.000103,-0.000917,-0.000638
liberall,-0.000107,0.000824,-0.000459,0.000520,-0.000586
barrons,0.000194,-0.000084,0.000471,-0.000471,0.000126
charismatic,-0.000482,-0.000091,0.001114,0.000654,0.000693
358,-0.000402,-0.000069,0.000011,-0.000660,0.000282
runnin,0.000206,0.000613,-0.000724,0.001594,0.001034
transparency,0.000231,0.000107,0.001195,-0.000828,0.000361
president,0.173807,-0.047690,0.341018,-0.016224,0.100096


In [52]:
short_text = [t[:50] for t in df["text"]]

In [53]:
#Calculating and printing  much does each topic apply to each tweet
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, columns=labels, index = short_text)
svd_topic_vectors_df.sample(40)

,topic0,topic1,topic2,topic3,topic4
Replying to \n@NBCPolitics\nWorking for Putin. \nWe,-0.124987,-0.070067,-0.013211,0.005478,0.062008
"""I fear that Putin is very likely to invade,"" Schi",0.103125,-0.043814,0.038158,-0.002889,-0.006661
Genocide in Ukraine: Vladimir Putin just gave his,0.107341,-0.067725,-0.061400,-0.015674,0.036024
Replying to \n@Mickangello\n and \n@IAPonomarenko\nwea,-0.002056,-0.041532,-0.027316,-0.008728,0.030199
Replying to \n@mark_b_goode\n @pointofsucces\n and 2,-0.077857,-0.009644,0.164212,0.069630,-0.032437
Ukraine and Poland used to be one country. My ance,0.076429,-0.051810,-0.031611,0.049165,-0.017586
I think the territorial claims argument is oversta,0.057420,0.064120,0.012939,0.003061,0.014292
Replying to \n@MisterChipT\nHitler did the same thin,-0.114835,-0.033034,0.010645,0.017606,-0.126828
Replying to \n@TheRickWilson\nHow long before we get,-0.071458,-0.017905,0.003672,-0.019615,-0.089763
Replying to \n@xrisii\nDid it ever occur to you that,0.027187,0.009684,0.098575,0.084761,0.022982


In [23]:
#Printing the highest scoring tweets for each topic
for topic in labels:
    print("________" + topic)
    print(df["text"].values[svd_topic_vectors_df[topic].argsort().values[-2:]])

________topic0
["Putin cannot be allowed to unilaterally expand Russia's territory in Eastern Europe. A Russian cyber attack on Ukraine is his first move. The U.S. must stand up to this dictator to prevent a Russian takeover of Ukraine.\nreuters.com\nCyberattack hits Ukraine as U.S. warns Russia could be prepping for war\nThe United States said it feared Russia was preparing a pretext to invade Ukraine if diplomacy fails to meet its objectives, after a massive cyberattack splashed Ukrainian government websites with a...\n2\n3\n7"
 'A message from Russians against the invasion of Ukraine \n\nWar is not Russia. War is Vladimir Putin and his government. That is why we, Russian socialists and communists, are against this criminal war\njacobin.com\nA Message From Russians Against the War\nWar is not Russia. War is Putin and his government. That is why we, Russian socialists and communists, are against this criminal war.\n2\n7\n11']
________topic1
["Putin has a whole puppet show going on.\nQ

In [24]:
df.iloc[0]["text"]

'Halt Russian aggression with 2-day shipping on ATGMs to Syrian rebels, Ukraine, Poland, Finland, and anyone else who wants to send a wire-guided “fuck Putin”\n3\n35'

In [25]:
svd_topic_vectors_df.iloc[0]

topic0    0.045870
topic1   -0.019814
topic2   -0.005545
topic3   -0.003467
topic4   -0.076924
Name: Halt Russian aggression with 2-day shipping on ATG, dtype: float64

In [26]:
#Printing the highest scoring words for each topic. 
num_terms = 20
for i in range(num_topics):
    print("___topic " + str(i) + "___")
    topicName = "topic" + str(i)
    weightedlist = topic_weights.get(topicName).sort_values()[-num_terms:]
    print(weightedlist.index.values)

___topic 0___
['nato' 'state' 'tweet' 'official' 'quote' '·' 'biden' 'ukrainian' 'say'
 'putin’s' 'invasion' 'u' 's' 'vladimir' 'com' 'president' 'russia' 'war'
 'russian' 'ukraine']
___topic 1___
['#demvoice1' '#ukraine' '3' '#putin' 'republican' 'twitter' 'feb' 'gas'
 'trump' 'price' 'mar' 'apr' 'jun' 'jan' '&' 'jul' 'thread' '·' 'quote'
 'tweet']
___topic 2___
['white' 'gop' 'russian' 'leader' 'house' 'meeting' 'election' 'joe' 'say'
 'republican' 'said' 'donald' 'com' 'u' 's' 'putin' 'vladimir' 'biden'
 'president' 'trump']
___topic 3___
['think' 'country' 'know' 'thread' '&' '2' 'like' 'just' 'biden' 'putin'
 '1' 'war' 'want' 'nato' '·' 'quote' 'tweet' 'russia' 'ukraine' 'trump']
___topic 4___
['joe' 'inflation' '9' 'ukrainian' 'price' 'crime' 'putin’s' '6' '5'
 'quote' '·' 'biden' 'tweet' 'vladimir' 'trump' 'russian' 'war'
 'president' '4' '1']


In [ ]:
#Applying Latent Dirichlet Allocation, using the code from the lecture notebook

In [27]:
#Calculating LDA on the Bag Of Words
count_vectoriser = CountVectorizer(tokenizer=my_tokeniser)
bag_of_words = count_vectoriser.fit_transform(df["text"])
vocab = count_vectoriser.get_feature_names_out()

print(bag_of_words.todense().shape)

/Users/kzaromytidou/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(10737, 40394)


In [28]:
lda = LatentDirichletAllocation(n_components=num_topics,
                                random_state=123,
                                learning_method='batch')

In [29]:
lda_topics = lda.fit_transform(bag_of_words)

In [30]:
#Calculate how  much does each topic apply to each tweet
lda_topic_vectors_df = pd.DataFrame(lda_topics, columns=labels)
lda_topic_vectors_df.sample(10)

,topic0,topic1,topic2,topic3,topic4
9568,0.004735,0.004720,0.500989,0.484858,0.004696
3319,0.008533,0.583892,0.008505,0.008644,0.390426
8664,0.445962,0.543628,0.003477,0.003478,0.003455
10537,0.006600,0.006731,0.702240,0.277835,0.006594
3538,0.018552,0.926025,0.018411,0.018648,0.018363
8647,0.005596,0.005636,0.005661,0.977603,0.005503
6566,0.015578,0.937647,0.015646,0.015537,0.015593
10730,0.003808,0.003767,0.308203,0.105147,0.579076
9710,0.010297,0.497554,0.010264,0.471578,0.010306
1308,0.005805,0.005876,0.976562,0.005931,0.005826


In [31]:
df["text"][4323]

'#Russia\nUkr Media:\n#Putin begins blackmailing Europe for #gas payments in #rubles. According to Reuters, gas pumping through the Yamal-Europe gas pipeline has fallen to zero.\n3\n4\n16'

In [32]:
#Calculating and printing the highest scoring reviews for each topic
for topic in labels:
    print(topic)
    #Get last songs 10 in list (highest scoring for each topic)
    print(df["text"].values[lda_topic_vectors_df[topic].argsort().values[-10:]])

topic0
['4 months later Sri Lanka has collapsed due to lack of fuel, Europe is back to burning coal to prepare for a dark winter, and people are starting to starve worldwide. \n\nHow many billions in lost  tax revenue now fuel Putin’s war machine \n@Tzeporah\n? \n\n#GreenLeapForward in action\nQuote Tweet\nTzeporah Berman\n@Tzeporah\n · Mar 9\nPlanet is quite literally, on fire, worlds leading scientists, economists, Nobel laureates etc call for immediate end expansion oil, gas & coal & absolute emissions and production decline & @globeandmail has an editorial on how world needs more Cdn oil & gas? Disgusted. #cdnpoli\n6\n19\n40'
 ".\n@POTUS\n announced at #NATO today, the U.S. is prepared to commit more than $1 billion in humanitarian assistance to help get relief to millions of Ukrainians affected by the war in Ukraine. We are #UnitedWithUkraine and the people in Ukraine in the face of Putin's aggression.\nQuote Tweet\nThe White House\n@WhiteHouse\n · Mar 24\nToday, the United States

In [33]:
#Calculating the most relevant tokens for each topic
for i, topic in enumerate(lda.components_):
    print("topic " + str(i) + ":")
    #Geting the  last n tokens (highest values)
    print(vocab[topic.argsort()[-num_terms:]])

topic 0:
['gas' '7' '6' "putin's" '&' '4' '2' 'nato' '1' 'russian' 'putin’s' '3'
 'com' 'quote' '·' 'tweet' 'war' 'russia' 'ukraine' 'putin']
topic 1:
['7' 'just' '4' '5' "putin's" "it's" '6' 'like' 'russian' '·' 'quote'
 'tweet' 'war' '3' 'russia' 'ukraine' '2' 'replying' '1' 'putin']
topic 2:
['just' '7' 'time' "putin's" 'like' '6' 'ukraine' '·' 'quote' 'biden'
 'tweet' '&' '3' '4' '5' '2' '1' 'trump' 'replying' 'putin']
topic 3:
['u' 's' "putin's" 'putin’s' 'invasion' 'ukrainian' 'say' '&' '3' '4' '2'
 'russia' 'president' 'vladimir' 'war' 'com' '1' 'ukraine' 'russian'
 'putin']
topic 4:
['know' 'trump' '8' '9' 'people' '&' 'like' 'ukraine' 'just' '5' '·'
 'quote' 'tweet' 'russia' '4' '1' '3' '2' 'replying' 'putin']
